In [1]:
# LeNet, AlexNet, and VGG all share a common design pattern: 
# extract features exploiting spatial structure via a sequence of convolutions and pooling layers and post-process the representations via fully connected layers. 
# The improvements upon LeNet by AlexNet and VGG mainly lie in how these later networks widen and deepen these two modules.


# This design poses two major challenges. First, the fully connected layers at the end of the architecture consume tremendous numbers of parameters. 
# For instance, even a simple model such as VGG-11 requires a monstrous matrix, occupying almost 400MB of RAM in single precision (FP32). 
# This is a significant impediment to computation, in particular on mobile and embedded devices. 
# After all, even high-end mobile phones sport no more than 8GB of RAM. At the time VGG was invented, 
# this was an order of magnitude less (the iPhone 4S had 512MB). As such, it would have been difficult to justify spending the majority of memory on an image classifier.

# Second, it is equally impossible to add fully connected layers earlier in the network to increase the degree of nonlinearity: 
# doing so would destroy the spatial structure and require potentially even more memory.

# The network in network (NiN) blocks (Lin et al., 2013) offer an alternative, capable of solving both problems in one simple strategy. 
# They were proposed based on a very simple insight: 
# (i) use 1*1 convolutions to add local nonlinearities across the channel activations and 
# (ii) use global average pooling to integrate across all locations in the last representation layer.
# Note that global average pooling would not be effective, were it not for the added nonlinearities. Let’s dive into this in detail.

'''8_1_deep_convolutional_nerual_networks_alexnet.ipynb
LeNet、AlexNet 和 VGG 都具有一个共同的设计模式：通过一系列卷积和池化层利用空间结构提取特征，然后通过全连接层对表示进行后处理。AlexNet 和 VGG 对 LeNet 的改进主要在于这些后来的网络如何加宽和加深这两个模块。

这种设计带来了两个主要挑战。首先，架构末端的全连接层消耗大量的参数。例如，即使是一个简单的模型，如 VGG-11，也需要一个巨大的矩阵，在单精度（FP32）下占用近 400MB 的 RAM。这对计算造成了很大的阻碍，特别是在移动和嵌入式设备上。毕竟，即使是高端手机也只有不超过 8GB 的 RAM。在 VGG 发明的时候，这个数字要少一个数量级（iPhone 4S 的 RAM 为 512MB）。因此，将大部分内存用于图像分类器是难以证明的。

其次，在网络的早期增加全连接层以增加非线性程度同样不可能：这样做会破坏空间结构，并可能需要更多的内存。

网络中的网络（NiN）块（Lin等人，2013）提供了一种替代方案，能够通过一种简单的策略解决这两个问题。它们是基于一个非常简单的见解提出的：（i）使用 1*1 卷积在通道激活之间添加局部非线性；（ii）在最后的表示层中使用全局平均池化在所有位置上进行整合。注意，如果没有添加非线性，全局平均池化将不会有效。让我们详细了解这个问题。

NiN 块的核心思想是在卷积层之间添加 1*1 卷积层，以增加跨通道的非线性变换。这使得网络可以在保持较低计算成本的同时，学习到更复杂的特征表示。然后，在网络的末端使用全局平均池化代替全连接层。全局平均池化将每个通道的特征图汇总为一个标量，从而显著减少了参数数量和计算成本。这种方法在保持网络性能的同时，降低了内存需求和计算复杂性，使得 NiN 更适合在移动和嵌入式设备上使用。

总之，NiN 通过在卷积层之间添加 1*1 卷积层以及在网络末端使用全局平均池化，解决了全连接层带来的内存和计算挑战。这使得网络能够在保持较高性能的同时，具有更低的计算和内存需求。
'''

import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def nin_block(out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.LazyConv2d(out_channels, kernel_size, strides, padding), nn.ReLU(),
        nn.LazyConv2d(out_channels, kernel_size=1), nn.ReLU(),
        nn.LazyConv2d(out_channels, kernel_size=1), nn.ReLU())

In [4]:
class NiN(d2l.Classifier):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
            nin_block(96, kernel_size=11, strides=4, padding=0),
            nn.MaxPool2d(3, stride=2),
            nin_block(256, kernel_size=5, strides=1, padding=2),
            nn.MaxPool2d(3, stride=2),
            nin_block(384, kernel_size=3, strides=1, padding=1),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout(0.5),
            nin_block(num_classes, kernel_size=3, strides=1, padding=1),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten())
        self.net.apply(d2l.init_cnn)

In [5]:
NiN().layer_summary((1, 1, 224, 224))

/data2/miniconda3/envs/d2l-pytorch/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


In [6]:
model = NiN(lr=0.05)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(224, 224))
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
trainer.fit(model, data)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### NiN的参数数量远远少于AlexNet和VGG。这主要是因为它不需要巨大的全连接层。
### 相反，它在网络主体的最后阶段之后，使用全局平均池化在所有图像位置上进行聚合。这消除了昂贵的（学习）降维操作的需要，并将它们替换为简单的平均。
### 当时令研究人员惊讶的事实是，这种平均操作并没有损害准确性。需要注意的是，在低分辨率表示（具有许多通道）上进行平均也增加了网络可以处理的平移不变性的数量。

### 选择较少的具有宽核的卷积并将它们替换为1*1卷积有助于进一步减少参数数量。它可以在任何给定位置的通道之间提供大量的非线性。1*1卷积和全局平均池化在很大程度上影响了后续的CNN设计。

### 通过减少参数数量，NiN能够在保持较高性能的同时降低计算和内存需求。1*1卷积可以在通道之间增加非线性，而全局平均池化可以在不损失准确性的情况下减少全连接层的需求。这
### 两种技术都对后续的卷积神经网络设计产生了深远影响，例如在GoogleNet（Inception模型）和ResNet中，它们都采用了1*1卷积和全局平均池化的思想。
### 这些网络在减少参数数量的同时，仍然保持了很高的性能和准确性，使它们在各种计算机视觉任务中取得了显著的成功。

1. 为什么每个NiN块有两个1*1卷积层？将它们的数量增加到三个。将它们的数量减少到一个。有什么变化？

每个NiN块中的两个1*1卷积层可以在通道之间引入更多的非线性。增加到三个1*1卷积层将进一步增加非线性，可能提高模型的表现能力，但也可能增加计算成本。将1*1卷积层数量减少到一个可能会降低非线性，这可能导致模型性能下降，但计算成本会减少。

2. 如果将1*1卷积替换为3*3卷积，会发生什么变化？

将1*1卷积替换为3*3卷积将增加模型的参数数量和计算成本。3*3卷积可以捕捉更大的感受野，可能提高模型的性能，但同时也可能增加过拟合的风险。

3. 如果将全局平均池化替换为全连接层（速度、准确性、参数数量），会发生什么？

将全局平均池化替换为全连接层会增加模型的参数数量和计算成本，可能导致训练速度变慢。此外，增加参数数量可能导致过拟合的风险增加，从而影响模型的泛化能力。然而，在某些情况下，全连接层可能提高模型的准确性。

4. 计算NiN的资源使用情况。

4.1 参数数量是多少？

NiN的参数数量取决于网络结构和层数。具体的参数数量需要根据实际使用的NiN模型来计算。

4.2 计算量是多少？

NiN的计算量主要取决于卷积层和全连接层的数量、卷积核大小以及输入输出通道数。具体的计算量需要根据实际使用的NiN模型来计算。

4.3 训练过程中需要多少内存？

训练过程中的内存需求取决于模型的参数数量、批量大小以及梯度的存储。具体的内存需求需要根据实际使用的NiN模型和训练设置来计算。

4.4 预测过程中需要多少内存？

预测过程中的内存需求主要取决于模型的参数数量和输入输出张量的大小。具体的内存需求需要根据实际使用的NiN模型来计算。

5. 将384*5*5表示一步减少到10 * 5 * 5表示可能存在哪些问题？

一步将384*5*5表示减少到10 * 5 * 5表示可能导致信息损失，从而影响模型的性能。此外，这种突然的降维可能导致梯度消失或梯度爆炸问题，从而影响训练过程的稳定性。

6. 使用在VGG中引入的结构设计决策，设计一系列类似NiN的网络。

可以参考VGG-11、VGG-16和VGG-19的设计思路，通过调整NiN块的数量、卷积层的数量和通道数来设计类似NiN的网络。例如，可以创建一个具有更多或更少NiN块的网络，或者在每个NiN块中增加或减少卷积层的数量。这些设计变化可以帮助探索不同